<a href="https://colab.research.google.com/github/TyrealQ/AI-Conversation/blob/main/LLM_Implementation/LLM_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Overview

This comprehensive guide provides code and detailed instructions for implementing OpenAI's GPT models across applications. Each section delivers clear explanations alongside practical code snippets, enabling effective LLM integration. The codebook serves both newcomers and experienced practitioners as a reference resource, helping users leverage language models efficiently to enhance analytical capabilities and streamline workflows.

Code authored by: **[Tyreal Qian](https://tyrealq.github.io/)**

## Install dependencies

In [ ]:
# %%capture
!pip install datasets
!pip install openai
!pip install tqdm

In [ ]:
# Interacting with the operating system
import os

# System-specific parameters and functions
import sys

# Handling JSON data
import json

# Data analysis and manipulation
import pandas as pd

# Handling large datasets efficiently
import datasets

# Loading datasets from Hugging Face
from datasets import load_dataset

# Displaying progress bars in loops
from tqdm import tqdm

# Handling dictionary default values efficiently
from collections import defaultdict

# Creating interactive widgets
import ipywidgets as widgets

# Display utilities for IPython environments
from IPython.display import display, JSON

# Interacting with OpenAI models
from openai import OpenAI

# Retrieving stored user credentials in Google Colab
from google.colab import userdata

# OpenAI API keys can be obtained at https://platform.openai.com
api_key = userdata.get('GPT_KEY')

# Initializing OpenAI client with the retrieved API key
client = OpenAI(api_key=api_key)

## Aspect-based sentiment analysis (ABSA) for game day experience

This project leverages LLMs as an innovative, off-the-shelf solution for ABSA in sports management. It enables scalable and efficient analysis of fan experiences without the complexities of traditional ABSA methods.



### Code and data are available in the [GitHub Repository](https://github.com/TyrealQ/Experience-is-all-you-need_SMR).

## TGL content classification with [GPT-4o](https://platform.openai.com/docs/models/gpt-4o)

This script automates the classification of YouTube video metadata to determine its relevance to the TMRW Golf League (TGL) using GPT-4o. It processes an input Excel file containing video details (title, description, and author) and assigns a filter rating based on predefined relevance criteria.

### Key Features:

- Utilize GPT-4o to analyze textual content and assign a rating from 1 (unrelated) to 5 (perfect match). Assign "MANUAL" for ambiguous cases requiring further review.
- Identify direct and indirect connections to TGL, including affiliated players, influencers, and events.
- Automatically assign a perfect match (5) if the video's author is @TGL.

### Co-developer: [Philip Kang](https://scholar.google.com/citations?user=VynUSnwAAAAJ&hl=en)

In [ ]:
# Load the dataset
ds = load_dataset("tyrealqian/TGL_content_classification")

# Check the dataset structure
print(ds)

In [ ]:
df = pd.DataFrame(ds['train'].to_pandas())
df

In [ ]:
def classify_document(doc_text):  # Define a function to classify a document based on its text input
    try:
        # Send document text to GPT-4o for classification
        response = client.chat.completions.create(
            model="gpt-4o",  # Use GPT-4o model for classification
            messages=[
                {
                    "role": "system",  # Define system instructions
                    "content": "You are an expert classifier.\
                    Your task is to evaluate whether the given document text is directly or contextually related to TMRW Golf League (TGL), an innovative golf league in partnership with the PGA TOUR that fuses advanced technology and live action in prime time.\
                    Content can be considered related if it explicitly mentions TGL or indirectly relates through associated players, influencers (e.g., Good Good Golf), events, or highlights (e.g., player shots, funny moments in TGL events).\
                    Special Rule: If the id_author of the video is @TGL, automatically assign a rating of 5 (perfect match) regardless of the content.\
                    Provide a single rating using the following scale:\
                    1 = no match (completely unrelated to TGL or its ecosystem),\
                    2 = weak match (mentions golf but no clear TGL connection),\
                    3 = moderate match (some connection, but TGL is not a key focus),\
                    4 = strong match (clear TGL connection but not exclusively about TGL),\
                    5 = perfect match (explicitly and primarily about TGL, or id_author equals @TGL),\
                    MANUAL = manual check required (unclear cases or possible indirect relevance).\
                    Output must be in valid JSON format with only the key filter_rating. Example: {\"filter_rating\": 4}."
                },
                {
                    "role": "user",  # Provide the document text as user input
                    "content": f"{doc_text}"
                }
            ],
            temperature=0,  # Ensure deterministic results
            max_tokens=2048,  # Allow sufficient token space for response
            top_p=1,  # Use deterministic sampling
            frequency_penalty=0,  # No penalty for frequent words
            presence_penalty=0,  # No encouragement for new topics
            response_format={"type": "json_object"}  # Enforce structured JSON output
        )

        # Parse and validate JSON response
        try:
            validation = json.loads(response.choices[0].message.content)  # Convert response to JSON
        except json.JSONDecodeError:
            print("Invalid JSON response. Defaulting to MANUAL.")  # Handle malformed responses
            validation = {"filter_rating": "MANUAL"}  # Assign "MANUAL" if parsing fails

        # Optional debugging prints
        print("\nDocument Preview:")
        print(f"{doc_text[:200]}...")  # Display first 200 characters of document text
        print("\nValidation Result:")
        print(json.dumps(validation, indent=2))  # Pretty-print classification result
        print("-" * 50)

        return json.dumps(validation)  # Return classification result as JSON string

    except Exception as e:
        print(f"\nError processing document: {e}")  # Handle unexpected errors
        return json.dumps({"filter_rating": "MANUAL"})  # Default to "MANUAL" in case of failure

# Read input Excel file
input_file = ds  # Path to input file
output_file = "/content/drive/MyDrive/DATA/TGL_classified.xlsx"  # Path to output file

try:
    print(f"Reading input file: {input_file}")
    df = pd.read_excel(input_file)  # Load the Excel file into a pandas DataFrame
except Exception as e:
    print(f"Error reading input file: {e}")
    exit(1)  # Exit if file read fails

# Classify each row with GPT-4o
print("\nStarting document classification...")
validations = []  # Initialize an empty list to store classification results
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Classifying Rows"):  # Iterate through rows with a progress bar
    if row['id_author'] == '@TGL':  # Auto-assign rating 5 if author is @TGL
        validations.append(json.dumps({"filter_rating": 5}))
    else:
        doc_text = f"{row['title']} {row['description']}"  # Combine title and description
        result_json_str = classify_document(doc_text)  # Get classification result
        validations.append(result_json_str)  # Store result

# Parse the returned JSON and store results
parsed_validations = [json.loads(v) for v in validations]  # Convert JSON strings to Python objects
df['filter_rating'] = [v.get('filter_rating', 'MANUAL') for v in parsed_validations]  # Extract ratings

# Save output to a new Excel file
try:
    df.to_excel(output_file, index=False)  # Save classified results to a new file
    print(f"\nClassification complete! Results saved to: {output_file}")
except Exception as e:
    print(f"Error saving output file: {e}")
    exit(1)  # Exit if file save fails

# Optional: Show summary of ratings
print("\nRating Summary:")
print(df['filter_rating'].value_counts(dropna=False))  # Display count of each rating category

Reading input file: DatasetDict({
    train: Dataset({
        features: ['video_link', 'title', 'lang', 'author', 'id_author', 'description', 'link_thumbnail', 'tt', 'views', 'likes', 'comments', 'duration', 'type_video'],
        num_rows: 33
    })
})
Error reading input file: Invalid file path or buffer object type: <class 'datasets.dataset_dict.DatasetDict'>

Starting document classification...


Classifying Rows:  12%|█▏        | 4/33 [00:01<00:09,  2.99it/s]


Document Preview:
TGL: Tiger Woods’ Revolutionary Golf League - Everything You Need to Know! Join us as we dive into TGL (Tomorrow’s Golf League), the exciting new golf league created by Tiger Woods! 🏌️‍♂️✨ In this vid...

Validation Result:
{
  "filter_rating": 5
}
--------------------------------------------------


Classifying Rows:  15%|█▌        | 5/33 [00:01<00:10,  2.56it/s]


Document Preview:
CLUTCH PUTT ⛳️ Justin Rose wins the hole for the Los Angeles Golf Club 🥶 | TGL on ESPN Watch as Justin Rose nails a 14-foot putt to give the Los Angeles Golf Club the win on hole 8 over Tiger Woods an...

Validation Result:
{
  "filter_rating": 5
}
--------------------------------------------------


...

Classifying Rows:  97%|█████████▋| 32/33 [00:24<00:00,  1.70it/s]


Document Preview:
The Good Good Hickory Clubs Major Thanks to SoFi for sponsoring the video! Click here to sign-up for SoFi Checking and Savings!
► https://sofi.com/goodgood

Our Apparel ► https://goodgoodgolf.com/

Th...

Validation Result:
{
  "filter_rating": 3
}
--------------------------------------------------


Classifying Rows: 100%|██████████| 33/33 [00:25<00:00,  1.31it/s]


Document Preview:
CURRENT AFFAIRS for BANKING EXAMS: 7th January, 2025 with SHOTS Stay updated with the most important Current Affairs for Banking Exams of 7th January, 2025  with SHOTS - Short Highlights of Top Storie...

Validation Result:
{
  "filter_rating": 1
}
--------------------------------------------------



Classification complete! Results saved to: /content/drive/MyDrive/DATA/TGL_classified.xlsx

Rating Summary:
filter_rating
5    20
4     5
2     5
1     2
3     1
Name: count, dtype: int64


In [ ]:
# Load the dataset
df1 = pd.read_excel("/content/drive/MyDrive/DATA/TGL_classified.xlsx")

# Select only the required columns
df1 = df1[["title", "id_author", "description", "filter_rating"]]

# Adjust pandas display settings to show full text
pd.set_option('display.max_colwidth', None)

# Sort ds1 by 'filter_rating' in descending order
df1 = df1.sort_values(by="filter_rating", ascending=False)

# Display the filtered and sorted DataFrame
df1

### Major Takeaways

- Use LLMs as intelligent assistants for scalable data analysis.

- Clear prompts and tools like Google Colab and the OpenAI API simplify complex filtering tasks.

- Structured outputs (e.g., JSON) and controlled parameters (temperature=0) ensure consistency and ease of integration.

- Include manual classification options for cases where automated methods fall short.


## Google Scholar publication extraction & ABDC ranking analysis

This script automates the extraction of publication details from Google Scholar content and evaluates them against the ABDC Journal Rankings. It processes scholarly content, extracts metadata, and performs ranking analysis to assess journal quality.

### Key Features:
- Extract publication titles, outlet names, and publication years from input text.
- Use GPT-4o to structure extracted data into JSON format.
- Compare extracted journals against the ABDC database to classify them by ranking (A\*, A, B, C).
- Load data from JSON and Excel files for systematic ranking analysis.
- Output matched/unmatched journals, ranking distribution, and validation checks.

In [ ]:
# File paths for Google Colab
JSON_PATH = "/content/drive/MyDrive/Teaching/Scholar/scholar_publications.json"
MARKDOWN_PATH = "/content/drive/MyDrive/Teaching/Scholar/ABDC_Q.md"

class ScholarAnalyzer:
    def __init__(self):
        self.scholar_content = None
        self.json_result = None
        self.publications = []

    def get_user_input(self):
        """Display text area for Google Scholar content"""
        text_area = widgets.Textarea(
            placeholder='Paste Google Scholar content here...',
            description='Scholar Content:',
            layout={'width': '100%', 'height': '300px'}
        )
        button = widgets.Button(description="Process Scholar Content")

        display(text_area)
        display(button)

        def on_button_clicked(b):
            self.scholar_content = text_area.value.strip()
            if not self.scholar_content:
                print("Error: No content provided")
                return

            button.description = "Processing..."
            button.disabled = True

            # Process the content
            self.process_scholar_content()

        button.on_click(on_button_clicked)

    def process_scholar_content(self):
        """Process Google Scholar content using OpenAI API"""
        print("Processing Google Scholar content...")

        try:
            client = OpenAI(api_key=api_key)

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "system",
                        "content": "Extract publications information from a Google Scholar page content and output in JSON format. Each publication should include the outlet name, article title, and publication year. Additionally, calculate and include the total number of publications.\n\nOutput Format:\n- JSON object with two fields:\n  - \"Publications\": A JSON list of objects, each with three fields: \"Outlet_Name\", \"Article_Title\", and \"Publication_Year\".\n  - \"Total_Publications\": A number representing the total count of publications."
                    },
                    {
                        "role": "user",
                        "content": self.scholar_content
                    }
                ],
                response_format={"type": "text"},
                temperature=0,
                max_tokens=16383
            )

            response_content = response.choices[0].message.content.strip()

            # Clean the JSON response
            if response_content.startswith("```json"):
                response_content = response_content[7:]
            if response_content.endswith("```"):
                response_content = response_content[:-3]

            response_content = response_content.strip()

            try:
                self.json_result = json.loads(response_content)
                print(f"Successfully extracted {self.json_result.get('Total_Publications', 0)} publications")

                # Display JSON preview
                print("\nJSON Preview:")
                display(JSON(self.json_result))

                # Save the result
                self.save_json_result()

                # Continue to ABDC analysis
                self.analyze_abdc_rankings()

            except json.JSONDecodeError as e:
                print(f"Error parsing JSON: {str(e)}")
                print("Raw response:\n", response_content[:500] + "...")

        except Exception as e:
            print(f"Error processing content: {str(e)}")

    def save_json_result(self):
        """Save JSON results to file"""
        try:
            # Ensure directory exists
            os.makedirs(os.path.dirname(JSON_PATH), exist_ok=True)

            with open(JSON_PATH, 'w', encoding='utf-8') as f:
                json.dump(self.json_result, f, indent=2, ensure_ascii=False)

            print(f"Results saved to {JSON_PATH}")

            # Verify file exists
            if os.path.exists(JSON_PATH):
                print("File verification successful ✓")
            else:
                print("Warning: File not found after saving!")

        except Exception as e:
            print(f"Error saving JSON: {str(e)}")

    def load_publications(self):
        """Load publications from JSON file"""
        try:
            with open(JSON_PATH, 'r', encoding='utf-8') as f:
                data = json.load(f)
                self.publications = data.get('Publications', [])
                print(f"Loaded {len(self.publications)} publications from file")
                return True
        except Exception as e:
            print(f"Error loading publications: {str(e)}")
            return False

    def load_abdc_rankings(self):
        """Load ABDC rankings from Markdown file"""
        try:
            # Read the markdown file
            with open(MARKDOWN_PATH, 'r', encoding='utf-8') as f:
                markdown_content = f.read()

            # Parse markdown table content
            # Assuming the markdown file contains a table with pipe separators
            lines = markdown_content.strip().split('\n')

            # Skip header and separator lines
            data_lines = [line for line in lines if '|' in line]
            if len(data_lines) > 2:  # Ensure we have header, separator, and at least one data row
                data_lines = data_lines[2:]  # Skip header and separator rows

            # Create dataframe
            journal_data = []
            for line in data_lines:
                cells = [cell.strip() for cell in line.split('|')]
                cells = [cell for cell in cells if cell]  # Remove empty cells from edges
                if len(cells) >= 2:  # Ensure we have at least journal name and ranking
                    journal_data.append({
                        'Journal_Name': cells[0],
                        'Ranking': cells[1]
                    })

            df = pd.DataFrame(journal_data)

            # Clean and standardize journal names and rankings
            df['Journal_Name'] = df['Journal_Name'].str.lower().str.strip()
            df['Ranking'] = df['Ranking'].str.strip()

            print(f"Loaded {len(df)} ABDC journal entries from markdown file")
            return df
        except Exception as e:
            print(f"Error loading ABDC rankings from markdown: {str(e)}")
            return pd.DataFrame()

    def save_analysis_to_markdown(self, results):
        """Save analysis results to markdown file"""
        try:
            output_path = "/content/drive/MyDrive/Teaching/Scholar/Analysis_results.md"

            summary = results['Summary']
            matched_journals = sorted(
                results['Matched_Journals'],
                key=lambda x: (x['Ranking'], -int(x['Year']) if str(x['Year']).isdigit() else 0)
            )
            unmatched_journals = sorted(
                results['Unmatched_Journals'],
                key=lambda x: -int(x['Year']) if str(x['Year']).isdigit() else 0
            )

            with open(output_path, 'w', encoding='utf-8') as f:
                # Title
                f.write("# ABDC Rankings Analysis\n\n")

                # Date
                from datetime import datetime
                current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                f.write(f"*Analysis generated on: {current_date}*\n\n")

                # Summary section
                f.write("## Summary\n\n")
                f.write("| Category | Count |\n")
                f.write("|----------|------:|\n")
                f.write(f"| Total Publications | {summary['Total_Publications']} |\n")
                f.write(f"| ABDC Listed Publications | {summary['Total_ABDC']} |\n")
                f.write(f"| A* Publications | {summary['A*']} |\n")
                f.write(f"| A Publications | {summary['A']} |\n")
                f.write(f"| B Publications | {summary['B']} |\n")
                f.write(f"| C Publications | {summary['C']} |\n")
                f.write(f"| Non-ABDC Publications | {summary['Non_ABDC']} |\n\n")

                # Matched journals section
                f.write("## ABDC Matched Publications\n\n")

                # Group by ranking
                current_ranking = None
                for journal in matched_journals:
                    if current_ranking != journal['Ranking']:
                        current_ranking = journal['Ranking']
                        f.write(f"### {current_ranking} Ranked Publications\n\n")

                    f.write(f"**{journal['Year']} - {journal['Journal']}**\n\n")
                    f.write(f"{journal['Title']}\n\n")

                # Unmatched journals section
                f.write("## Non-ABDC Publications\n\n")
                for journal in unmatched_journals:
                    f.write(f"**{journal['Year']} - {journal['Journal']}**\n\n")
                    f.write(f"{journal['Title']}\n\n")

            print(f"\nAnalysis saved to markdown file: {output_path}")
            return True
        except Exception as e:
            print(f"Error saving analysis to markdown: {str(e)}")
            return False

    def analyze_abdc_rankings(self):
        """Analyze publications against ABDC rankings"""
        print("\nAnalyzing publications against ABDC rankings...")

        # If we're coming from JSON processing, use the publications from there
        if not self.publications and self.json_result:
            self.publications = self.json_result.get('Publications', [])

        # Otherwise try to load from file
        if not self.publications:
            if not self.load_publications():
                print("No publications to analyze. Please process Google Scholar content first.")
                return

        # Load ABDC rankings
        rankings_df = self.load_abdc_rankings()
        if rankings_df.empty:
            print("Failed to load ABDC rankings from markdown file")
            return

        # Create results structure
        results = {
            'Total_Publications': len(self.publications),
            'Rankings': defaultdict(int, {
                'A*': 0, 'A': 0, 'B': 0, 'C': 0, 'Non-ABDC': 0
            }),
            'Matched_Journals': [],
            'Unmatched_Journals': []
        }

        # Create lookup dictionary for faster matching
        rankings_lookup = dict(zip(rankings_df['Journal_Name'], rankings_df['Ranking']))

        # Match publications with rankings
        for pub in self.publications:
            matched, ranking = self.find_journal_match(pub['Outlet_Name'], rankings_lookup)

            if matched:
                # Count the ranking
                results['Rankings'][ranking] += 1

                # Add to matched journals list
                results['Matched_Journals'].append({
                    'Journal': pub['Outlet_Name'],
                    'Ranking': ranking,
                    'Year': pub['Publication_Year'],
                    'Title': pub['Article_Title']
                })
            else:
                results['Rankings']['Non-ABDC'] += 1
                results['Unmatched_Journals'].append({
                    'Journal': pub['Outlet_Name'],
                    'Year': pub['Publication_Year'],
                    'Title': pub['Article_Title']
                })

        # Calculate summary statistics
        total_abdc = (results['Rankings']['A*'] +
                      results['Rankings']['A'] +
                      results['Rankings']['B'] +
                      results['Rankings']['C'])

        results['Summary'] = {
            'Total_Publications': results['Total_Publications'],
            'Total_ABDC': total_abdc,
            'A*': results['Rankings']['A*'],
            'A': results['Rankings']['A'],
            'B': results['Rankings']['B'],
            'C': results['Rankings']['C'],
            'Non_ABDC': results['Rankings']['Non-ABDC']
        }

        # Print the analysis report
        self.print_analysis_report(results)

    def find_journal_match(self, journal_name, rankings_lookup):
        """Find if a journal matches any in the ABDC list with 100% match requirement"""
        journal_name_lower = journal_name.lower().strip()

        # Try exact match only (case-insensitive)
        if journal_name_lower in rankings_lookup:
            ranking = rankings_lookup[journal_name_lower].strip()
            return True, ranking

        # No partial matching or special cases anymore
        return False, None

    def print_analysis_report(self, results):
        """Print detailed analysis report and save to markdown file"""
        summary = results['Summary']

        # Print to console
        print("\nABDC Rankings Analysis:")
        print("-----------------------")
        print(f"Total Publications: {summary['Total_Publications']}")
        print(f"ABDC Listed Publications: {summary['Total_ABDC']}")
        print(f"A* Publications: {summary['A*']}")
        print(f"A Publications: {summary['A']}")
        print(f"B Publications: {summary['B']}")
        print(f"C Publications: {summary['C']}")
        print(f"Non-ABDC Publications: {summary['Non_ABDC']}")

        print("\nMatched ABDC Journals:")
        print("----------------------")
        # Sort by ranking and year
        matched_journals = sorted(
            results['Matched_Journals'],
            key=lambda x: (x['Ranking'], -int(x['Year']) if str(x['Year']).isdigit() else 0)
        )

        # Group by ranking
        current_ranking = None
        for journal in matched_journals:
            if current_ranking != journal['Ranking']:
                current_ranking = journal['Ranking']
                print(f"\n{current_ranking} Ranked Publications:")
                print("-" * (len(current_ranking) + 20))

            print(f"{journal['Year']} - {journal['Journal']}")
            print(f"Title: {journal['Title']}")

        print("\nUnmatched Journals:")
        print("-----------------")
        unmatched_journals = sorted(
            results['Unmatched_Journals'],
            key=lambda x: -int(x['Year']) if str(x['Year']).isdigit() else 0
        )
        for journal in unmatched_journals:
            print(f"{journal['Year']} - {journal['Journal']}")
            print(f"Title: {journal['Title']}")

        print("\nAnalysis complete!")

        # Save to markdown file
        self.save_analysis_to_markdown(results)

# Run the analyzer
analyzer = ScholarAnalyzer()

# Option to run just the ABDC analysis (if JSON file already exists)
run_only_analysis = False  # Set to True to skip Scholar content extraction

if run_only_analysis:
    analyzer.analyze_abdc_rankings()
else:
    analyzer.get_user_input()

## Next: Develop a custom multi-agent system for your work